<a href="https://colab.research.google.com/github/AfrinDange/attendance_analysis/blob/main/scripts/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Git Repository

In [ ]:
!git clone https://github.com/AfrinDange/attendance_analysis.git

fatal: destination path 'attendance_analysis' already exists and is not an empty directory.


# Importing Libraries

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime
import csv
import xlrd
import json
import uuid

# Roll Numbers

In [ ]:
roll_no = pd.read_excel("attendance_analysis/data/roll_no.xlsx")
display(roll_no)
roll_no = roll_no.set_index('user_name').T.to_dict('roll_no')[0]

,roll_no,user_name
0,101710,CHRISTOPHER CARDOZA
1,101775,joel.thomas
2,101801,ALWIN JACOB
3,101802,B SAI SREE
4,101803,B SHRIRAM ATHREYA
...,...,...
67,101867,rahul.khambe
68,101868,vedesh.nadar
69,101869,shawn.periera
70,101870,gargi raje


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


# Sample Data

In [ ]:
sample = pd.read_csv("attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3).csv", encoding = "UTF-16", skiprows=6, sep='\t')
display(sample)

,Full Name,Join Time,Leave Time,Duration,userPrincipalName,Role
0,Smita Dange,"8/12/2021, 4:32:58 PM","8/12/2021, 5:02:52 PM",29m 54s,smita.dange@fcrit.ac.in,Organizer
1,Smita Dange,"8/12/2021, 5:05:31 PM","8/12/2021, 5:32:29 PM",26m 57s,smita.dange@fcrit.ac.in,Organizer
2,rahul.khambe,"8/12/2021, 4:33:15 PM","8/12/2021, 5:31:33 PM",58m 17s,rahul.khambe@FCRIT.onmicrosoft.com,Presenter
3,PULHANI RACHIT,"8/12/2021, 4:33:35 PM","8/12/2021, 5:32:34 PM",58m 58s,pulhani.rachit@FCRIT.onmicrosoft.com,Presenter
4,BHAGYASHREE LAKSHMINARAYANAN,"8/12/2021, 4:33:36 PM","8/12/2021, 5:37:34 PM",1h 3m,bhagyashree.lakshmi@FCRIT.onmicrosoft.com,Presenter
...,...,...,...,...,...,...
69,THAKUR YUKTA,"8/12/2021, 4:43:26 PM","8/12/2021, 5:33:05 PM",49m 38s,thakur.yukta@FCRIT.onmicrosoft.com,Presenter
70,vedesh.nadar,"8/12/2021, 4:53:20 PM","8/12/2021, 5:44:01 PM",50m 41s,vedesh.nadar@FCRIT.onmicrosoft.com,Presenter
71,CHAVAN SHREYA,"8/12/2021, 4:56:25 PM","8/12/2021, 5:41:04 PM",44m 38s,chavan.shreya@FCRIT.onmicrosoft.com,Presenter
72,GUPTA ISHIKA,"8/12/2021, 5:13:32 PM","8/12/2021, 5:35:15 PM",21m 43s,gupta.ishika@FCRIT.onmicrosoft.com,Presenter


In [ ]:
sample = pd.read_excel("attendance_analysis/data/MCC/MCC L23 26-08-2021.xlsx", skiprows=7)
display(sample)

,Full Name,Join Time,Leave Time,Duration,userPrincipalName,Role
0,Rakhi Kalantri,"8/26/2021, 10:11:23 AM","8/26/2021, 11:17:03 AM",1h 5m,rakhi.kalantri@fcrit.ac.in,Organizer
1,DESHMUKH PARTH,"8/26/2021, 10:15:35 AM","8/26/2021, 11:17:03 AM",1h 1m,deshmukh.parth@comp.fcrit.ac.in,Attendee
2,SERENA RAJU,"8/26/2021, 10:15:51 AM","8/26/2021, 11:17:03 AM",1h 1m,serena.raju@comp.fcrit.ac.in,Attendee
3,GEOFERRY MICHAEL,"8/26/2021, 10:16:41 AM","8/26/2021, 11:17:03 AM",1h,geoferry.michael@comp.fcrit.ac.in,Attendee
4,HUDIYA YASH,"8/26/2021, 10:16:48 AM","8/26/2021, 11:17:03 AM",1h,hudiya.yash@comp.fcrit.ac.in,Attendee
...,...,...,...,...,...,...
68,pramod more,"8/26/2021, 10:55:45 AM","8/26/2021, 10:55:56 AM",10s,pramod.more@fcrit.ac.in,Attendee
69,joel.thomas,"8/26/2021, 10:36:09 AM","8/26/2021, 11:17:03 AM",40m 54s,joel@FCRIT.onmicrosoft.com,Attendee
70,GUPTA CHIRANYA,"8/26/2021, 10:36:41 AM","8/26/2021, 11:17:03 AM",40m 22s,gupta.chiranya@comp.fcrit.ac.in,Attendee
71,GEORGE SIBIN,"8/26/2021, 10:39:05 AM","8/26/2021, 11:17:03 AM",37m 58s,george.sibin@comp.fcrit.ac.in,Attendee


# Convert Duration to Seconds

In [ ]:
def time_parser(time):
    num_split = re.split("h|m|s", time)
    h_ind = time.find("h")
    m_ind = time.find("m")
    s_ind = time.find("s")

    secs = 0

    if h_ind != -1: # xh ... 
       secs += int(num_split[0])*60*60
    if m_ind != -1:
        if h_ind == -1: # xm ...
            secs += int(num_split[0])*60
        else: # xh ym ...
            secs += int(num_split[1])*60
    if s_ind != -1:
        if h_ind == -1: 
            if m_ind == -1: # xs
                secs += int(num_split[0])
            else: # xm ys
                secs += int(num_split[1])
        else: 
            if m_ind == -1: # xh ys
                secs += int(num_split[1])
            else: # xh ym zs
                secs += int(num_split[2])
    return secs

# Test Script

In [69]:
subject = "DSIP"
dirPath = "attendance_analysis/data/"
all_data = []
for file in os.listdir(dirPath+subject):
    lecture_data = {}
    lecture_data['subject'] = subject
    if ".csv" in file:
        with open(dirPath+subject+"/"+file,'rt', encoding='utf16')as f:
            data = csv.reader(f, delimiter="\t")
            i = 0
            for row in data:
                if len(row) == 0:
                    skipped = i
                    break
                # if row[0] == 'Total Number of Participants':
                #     lecture_data['participants'] = int(row[1])
                # if row[0] == 'Meeting Title':
                #     lecture_data['meeting_title'] = row[1]
                if row[0] == 'Meeting Start Time':
                    lecture_data['meeting_start_time'] = row[1]
                if row[0] == 'Meeting End Time':
                    lecture_data['meeting_end_time'] = row[1]
                i = i + 1
        print(dirPath+subject+"/"+file)      
        lecture = pd.read_csv(dirPath+subject+"/"+file, encoding = "UTF-16", skiprows=skipped, sep="\t")
    else:
        wb = xlrd.open_workbook(dirPath+subject+"/"+file)
        sheet = wb.sheet_by_index(0)
        skipped = 0
        for i in range(0,sheet.nrows):
            row = sheet.row_values(i) 
            # if row[0] == 'Total Number of Participants':
            #     lecture_data['participants'] = int(row[1])
            # if row[0] == 'Meeting Title':
            #     lecture_data['meeting_title'] = row[1]
            if row[0] == 'Meeting Start Time':
                lecture_data['meeting_start_time'] = row[1]
            if row[0] == 'Meeting End Time':
                lecture_data['meeting_end_time'] = row[1]
            if row[0] == '':
                skipped = i+1
                break
        print(dirPath+subject+"/"+file)
        lecture = pd.read_excel(dirPath+subject+"/"+file, skiprows=skipped)
    lecture_data['lecture_id'] = str(uuid.uuid4())
    attendees = {}
    organizer = {}
    for index,row in lecture.iterrows():
        # if row['Full Name'] == 'pramod more':
        #     lecture_data['attendance_form_time'] = row['Join Time']
        #     break
        if row['Full Name'] not in attendees.keys():
            if row['Role'] == 'Presenter' or row['Role'] == 'Attendee':
                if row['Full Name'] in roll_no.keys():
                    attendee = {}
                    attendee['name'] = row['Full Name']
                    attendee['roll_no'] = roll_no[row['Full Name']]
                    attendee['join_time'] = row['Join Time']
                    attendee['leave_time'] = row['Leave Time']
                    attendee['duration'] = time_parser(row['Duration'])
                    # attendee['role'] = row['Role']
                    try:
                        attendee['email'] = row['Email']
                    except KeyError:
                        try: 
                            attendee['email'] = row['Email (userPrincipalName)']
                        except KeyError:
                            attendee['email'] = row['userPrincipalName']
                    if type(attendee['email']) is float:
                        attendee['email'] = "-"
                    attendees[row['Full Name']] = attendee
            else:
                if not bool(organizer):
                    organizer['organizer_name'] = row['Full Name']
                    organizer['organizer_join_time'] = row['Join Time']
                    organizer['organizer_leave_time'] = row['Leave Time']
                    organizer['organizer_duration'] = time_parser(row['Duration'])
                else:
                    prev_join_time = datetime.strptime(organizer['organizer_join_time'], '%m/%d/%Y, %I:%M:%S %p')
                    curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
                    if prev_join_time < curr_join_time: 
                        organizer['organizer_leave_time'] = row['Leave Time']
                    else:
                        organizer['organizer_join_time'] = row['Join Time']
                    organizer['organizer_duration'] += time_parser(row['Duration'])
        else: # multiple join-leave
            attendee = attendees[row['Full Name']]
            prev_join_time = datetime.strptime(attendee['join_time'], '%m/%d/%Y, %I:%M:%S %p')
            curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
            if prev_join_time < curr_join_time: 
                attendee['leave_time'] = row['Leave Time']
            else:
                attendee['join_time'] = row['Join Time']
            attendee['duration'] += time_parser(row['Duration'])
    for key, student in attendees.items():
        final = {**student, **lecture_data, **organizer}
        all_data.append(final)

attendance_analysis/data/DSIP/4-8-2021.csv
attendance_analysis/data/DSIP/25-8-2021 L no. 25.csv
attendance_analysis/data/DSIP/31-8-2021 Lno 28.csv
attendance_analysis/data/DSIP/17-8-2021-L no 22,23.csv
attendance_analysis/data/DSIP/5-8-2021.csv
attendance_analysis/data/DSIP/11-8-2021.csv
attendance_analysis/data/DSIP/16-8-2021-L no 21.csv
attendance_analysis/data/DSIP/24-8-2021- Lno 24.csv
attendance_analysis/data/DSIP/3-8-2021.csv
attendance_analysis/data/DSIP/26-8-2021 L no 26.csv
attendance_analysis/data/DSIP/6-8-2021.csv
attendance_analysis/data/DSIP/10-8-2021.csv


In [70]:
preprocessed_dirPath = 'attendance_analysis/preprocessed_data_json/'

# Preprocessing Data

In [71]:
def preprocessor(subject):
    dirPath = "attendance_analysis/data/"
    all_data = []
    for file in os.listdir(dirPath+subject):
        lecture_data = {}
        lecture_data['subject'] = subject
        if ".csv" in file:
            with open(dirPath+subject+"/"+file,'rt', encoding='utf16')as f:
                data = csv.reader(f, delimiter="\t")
                i = 0
                for row in data:
                    if len(row) == 0:
                        skipped = i
                        break
                    # if row[0] == 'Total Number of Participants':
                    #     lecture_data['participants'] = int(row[1])
                    # if row[0] == 'Meeting Title':
                    #     lecture_data['meeting_title'] = row[1]
                    if row[0] == 'Meeting Start Time':
                        lecture_data['meeting_start_time'] = row[1]
                    if row[0] == 'Meeting End Time':
                        lecture_data['meeting_end_time'] = row[1]
                    i = i + 1
            print(dirPath+subject+"/"+file)      
            lecture = pd.read_csv(dirPath+subject+"/"+file, encoding = "UTF-16", skiprows=skipped, sep="\t")
        else:
            wb = xlrd.open_workbook(dirPath+subject+"/"+file)
            sheet = wb.sheet_by_index(0)
            skipped = 0
            for i in range(0,sheet.nrows):
                row = sheet.row_values(i) 
                # if row[0] == 'Total Number of Participants':
                #     lecture_data['participants'] = int(row[1])
                # if row[0] == 'Meeting Title':
                #     lecture_data['meeting_title'] = row[1]
                if row[0] == 'Meeting Start Time':
                    lecture_data['meeting_start_time'] = row[1]
                if row[0] == 'Meeting End Time':
                    lecture_data['meeting_end_time'] = row[1]
                if row[0] == '':
                    skipped = i+1
                    break
            print(dirPath+subject+"/"+file)
            lecture = pd.read_excel(dirPath+subject+"/"+file, skiprows=skipped)
        lecture_data['lecture_id'] = str(uuid.uuid4())
        attendees = {}
        organizer = {}
        for index,row in lecture.iterrows():
            if row['Full Name'] == 'pramod more':
                lecture_data['attendance_form_time'] = row['Join Time']
                break
            if row['Full Name'] not in attendees.keys():
                if row['Role'] == 'Presenter' or row['Role'] == 'Attendee':
                    if row['Full Name'] in roll_no.keys():
                        attendee = {}
                        attendee['name'] = row['Full Name']
                        attendee['roll_no'] = roll_no[row['Full Name']]
                        attendee['join_time'] = row['Join Time']
                        attendee['leave_time'] = row['Leave Time']
                        attendee['duration'] = time_parser(row['Duration'])
                        # attendee['role'] = row['Role']
                        try:
                            attendee['email'] = row['Email']
                        except KeyError:
                            try: 
                                attendee['email'] = row['Email (userPrincipalName)']
                            except KeyError:
                                attendee['email'] = row['userPrincipalName']
                        if type(attendee['email']) is float:
                            attendee['email'] = "-"
                        attendees[row['Full Name']] = attendee
                else:
                    if not bool(organizer):
                        organizer['organizer_name'] = row['Full Name']
                        organizer['organizer_join_time'] = row['Join Time']
                        organizer['organizer_leave_time'] = row['Leave Time']
                        organizer['organizer_duration'] = time_parser(row['Duration'])
                    else:
                        prev_join_time = datetime.strptime(organizer['organizer_join_time'], '%m/%d/%Y, %I:%M:%S %p')
                        curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
                        if prev_join_time < curr_join_time: 
                            organizer['organizer_leave_time'] = row['Leave Time']
                        else:
                            organizer['organizer_join_time'] = row['Join Time']
                        organizer['organizer_duration'] += time_parser(row['Duration'])
            else: # multiple join-leave
                attendee = attendees[row['Full Name']]
                prev_join_time = datetime.strptime(attendee['join_time'], '%m/%d/%Y, %I:%M:%S %p')
                curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
                if prev_join_time < curr_join_time: 
                    attendee['leave_time'] = row['Leave Time']
                else:
                    attendee['join_time'] = row['Join Time']
                attendee['duration'] += time_parser(row['Duration'])
        for key, student in attendees.items():
            final = {**student, **lecture_data, **organizer}
            all_data.append(final)
    json_data = json.dumps(all_data, indent = 2)
    with open(preprocessed_dirPath+subject+"_data.json", "w") as outfile:
        outfile.write(json_data)
    print("\n\nFiled Created at -> ", preprocessed_dirPath+subject+"_data.json")

# Saving Data

In [ ]:
bda_data = preprocessor("BDA")

attendance_analysis/data/BDA/meetingAttendanceReport(General) (2).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (2).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (5).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1) (2).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (4).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (2) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(General).csv
attendance_analysis/data/BDA/meetingAttendanceReport(General) (3).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(General) (1).csv


Filed Created at ->  attendance_analysis

In [ ]:
dsip_data = preprocessor("DSIP")

attendance_analysis/data/DSIP/4-8-2021.csv
attendance_analysis/data/DSIP/25-8-2021 L no. 25.csv
attendance_analysis/data/DSIP/31-8-2021 Lno 28.csv
attendance_analysis/data/DSIP/17-8-2021-L no 22,23.csv
attendance_analysis/data/DSIP/5-8-2021.csv
attendance_analysis/data/DSIP/11-8-2021.csv
attendance_analysis/data/DSIP/16-8-2021-L no 21.csv
attendance_analysis/data/DSIP/24-8-2021- Lno 24.csv
attendance_analysis/data/DSIP/3-8-2021.csv
attendance_analysis/data/DSIP/26-8-2021 L no 26.csv
attendance_analysis/data/DSIP/6-8-2021.csv
attendance_analysis/data/DSIP/10-8-2021.csv


Filed Created at ->  attendance_analysis/preprocessed_data_json/DSIP_data.json


In [72]:
mcc_data = preprocessor("MCC")

attendance_analysis/data/MCC/MCC L23 26-08-2021.xlsx
attendance_analysis/data/MCC/MCC L31 16-09-2021.xlsx
attendance_analysis/data/MCC/MCC L12 29-07-2021.xlsx
attendance_analysis/data/MCC/MCC L2.xlsx
attendance_analysis/data/MCC/MCC L7.xlsx
attendance_analysis/data/MCC/MCC L28 08-09-2021.xlsx
attendance_analysis/data/MCC/MCC L33 17-09-2021.xlsx
attendance_analysis/data/MCC/MCC L22 25-08-2021.xlsx
attendance_analysis/data/MCC/MCC L14 04-08-2021.xlsx
attendance_analysis/data/MCC/MCC L24 27-08-2021.xlsx
attendance_analysis/data/MCC/MCC L9.xlsx
attendance_analysis/data/MCC/MCC L21 13-08-2021.xlsx
attendance_analysis/data/MCC/MCC L35 22-09-2021.xlsx
attendance_analysis/data/MCC/MCC L16 06-08-2021.xlsx
attendance_analysis/data/MCC/MCC L27 06-09-2021.xlsx
attendance_analysis/data/MCC/MCC L5.xlsx
attendance_analysis/data/MCC/MCC L18 10-08-2021.xlsx
attendance_analysis/data/MCC/MCC L17 09-08-2021.xlsx
attendance_analysis/data/MCC/MCC L10.xlsx
attendance_analysis/data/MCC/MCC L6.xlsx
attendance_